<a href="https://colab.research.google.com/github/msarkar2501/YOLO_sample/blob/main/Run_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Manit Sarkar          
Roll Number: 102308157     
Group: 2H2C

INSTALL LIBRARIES

In [ ]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.7 MB/s eta 0:00:00


In [8]:
import os
from google.colab import drive
from ultralytics import YOLO
import torch
import torchvision
from pathlib import Path
import shutil

CHECK IF GPU IS AVAILABLE

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

Running on: cuda


CREATE WORKING DIRECTORY

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define project root in Drive
BASE_DIR = '/content/drive/MyDrive/yolov8_project'
os.makedirs(BASE_DIR, exist_ok=True)

# Optional: change working directory
%cd $BASE_DIR

# Create datasets folder
DATASET_DIR = f"{BASE_DIR}/datasets/VOC"
os.makedirs(DATASET_DIR, exist_ok=True)

# Redirect 'runs' folder to Drive
!rm -rf /content/runs
os.makedirs(f"{BASE_DIR}/runs", exist_ok=True)
!ln -s "{BASE_DIR}/runs" /content/runs

Mounted at /content/drive
/content/drive/MyDrive/yolov8_project


DOWNLOAD AND CONVERT THE DATASET

In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path
from tqdm import tqdm
from ultralytics.utils.downloads import download

# Base directory in Google Drive
base_dir = Path("/content/drive/MyDrive/yolov8_project")

yaml = {
    "path": str(base_dir / "datasets" / "VOC"),
    "names": {
        0: "aeroplane", 1: "bicycle", 2: "bird", 3: "boat", 4: "bottle",
        5: "bus", 6: "car", 7: "cat", 8: "chair", 9: "cow",
        10: "diningtable", 11: "dog", 12: "horse", 13: "motorbike",
        14: "person", 15: "pottedplant", 16: "sheep", 17: "sofa",
        18: "train", 19: "tvmonitor"
    }
}

def convert_label(path, lb_path, year, image_id):
    def convert_box(size, box):
        dw, dh = 1.0 / size[0], 1.0 / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(path / f"VOC{year}/Annotations/{image_id}.xml")
    out_file = open(lb_path, "w")
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)
    names = list(yaml["names"].values())

    for obj in root.iter("object"):
        cls = obj.find("name").text
        if cls in names and int(obj.find("difficult").text) != 1:
            xmlbox = obj.find("bndbox")
            bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ("xmin", "xmax", "ymin", "ymax")])
            cls_id = names.index(cls)
            out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + "\n")

# Download and unzip dataset to Drive
dir = base_dir / "datasets" / "VOC"
url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/"
urls = [
    f"{url}VOCtrainval_06-Nov-2007.zip",
    f"{url}VOCtest_06-Nov-2007.zip",
    f"{url}VOCtrainval_11-May-2012.zip",
]
download(urls, dir=dir / "images", curl=True, threads=3, exist_ok=True)

# Convert VOC to YOLO format and move images & labels inside Drive
path = dir / "images" / "VOCdevkit"
for year, image_set in ("2012", "train"), ("2012", "val"), ("2007", "train"), ("2007", "val"), ("2007", "test"):
    imgs_path = dir / "images" / f"{image_set}{year}"
    lbs_path = dir / "labels" / f"{image_set}{year}"
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    with open(path / f"VOC{year}/ImageSets/Main/{image_set}.txt") as f:
        image_ids = f.read().strip().split()
    for id in tqdm(image_ids, desc=f"{image_set}{year}"):
        img_file = path / f"VOC{year}/JPEGImages/{id}.jpg"
        lb_file = (lbs_path / img_file.name).with_suffix(".txt")
        img_file.rename(imgs_path / img_file.name)
        convert_label(path, lb_file, year, id)

print("✅ VOC dataset downloaded and converted to YOLO format.")

test2007: 100%|██████████| 4952/4952 [01:02<00:00, 79.87it/s]

✅ VOC dataset downloaded and converted to YOLO format.


CREATE THE DATA.YAML FILE FOR EASY MAPPING

In [ ]:
# Write data.yaml inside Drive
yaml_content = f"""
path: {dir}

train:
  - images/train2012
  - images/val2012

val:
  - images/test2007

test:
  - images/test2007

names:
  0: aeroplane
  1: bicycle
  2: bird
  3: boat
  4: bottle
  5: bus
  6: car
  7: cat
  8: chair
  9: cow
  10: diningtable
  11: dog
  12: horse
  13: motorbike
  14: person
  15: pottedplant
  16: sheep
  17: sofa
  18: train
  19: tvmonitor
"""

with open(dir / "data.yaml", "w") as f:
    f.write(yaml_content.strip())

print(f"✅ data.yaml file created at {dir / 'data.yaml'}")

✅ data.yaml file created at /content/drive/MyDrive/yolov8_project/datasets/VOC/data.yaml


TRAIN THE MODEL

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data=str(base_dir / "datasets" / "VOC" / "data.yaml"),
    epochs=20,
    imgsz=640,
    batch=16,
    name='train_1',
    workers=4,
    project=str(base_dir),       # Save runs here inside Drive
    exist_ok=True
)


100%|██████████| 6.25M/6.25M [00:00<00:00, 105MB/s]


Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolov8_project/datasets/VOC/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

100%|██████████| 755k/755k [00:00<00:00, 24.5MB/s]

Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

 22        [15, 18, 21]  1    755212  ultralytics.nn.modules.head.Detect           [20, [64, 128, 256]]          
Model summary: 129 layers, 3,014,748 parameters, 3,014,732 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

      20/20      3.64G     0.7706     0.7872      1.129         11        640: 100%|██████████| 722/722 [03:53<00:00,  3.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:51<00:00,  3.01it/s]


                   all       4952      12032      0.782      0.738      0.805      0.593

20 epochs completed in 1.691 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8_project/train_1/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/yolov8_project/train_1/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/yolov8_project/train_1/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:56<00:00,  2.73it/s]


                   all       4952      12032      0.783      0.738      0.805      0.593
             aeroplane        204        285      0.893      0.789      0.884      0.643
               bicycle        239        337      0.907      0.742      0.888      0.654
                  bird        282        459        0.8      0.679      0.789       0.53
                  boat        172        263      0.697      0.631      0.707      0.463
                bottle        212        469      0.762      0.689      0.718       0.49
                   bus        174        213      0.865      0.785      0.892      0.768
                   car        721       1201      0.852      0.865      0.917       0.73
                   cat        322        358      0.836      0.812      0.876      0.684
                 chair        417        756       0.59      0.605      0.602      0.411
                   cow        127        244      0.751      0.717        0.8      0.594
           diningtabl

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b9446faf350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043

VALIDATE THE EVALUATE THE MODEL

In [10]:
# Use existing base directory
base_dir = Path("/content/drive/MyDrive/yolov8_project")

# Load the trained model
model = YOLO(base_dir / "train_1/weights/best.pt")

# ✅ 1. VALIDATION
results = model.val(
    data=str(base_dir / "datasets" / "VOC" / "data.yaml"),
    save=True,
    save_dir=str(base_dir / "val_1")
)
print("✅ Validation complete. Saved at:", base_dir / "val_1")

# ✅ 2. EVALUATION ON SAMPLE IMAGES
evaluate = model.predict(
    source="/content/drive/MyDrive/sample_images",
    save=True,
    name="eval_1",       # will be created at /content/runs/detect/eval_1
    save_txt=True
)

# Move eval results to Drive
eval_src = Path("/content/runs/detect/eval_1")
eval_dst = base_dir / "eval_1"

# Clear existing destination if it exists
if eval_dst.exists():
    shutil.rmtree(eval_dst)

shutil.move(str(eval_src), str(eval_dst))

print("✅ Evaluation on sample images complete. Saved at:", eval_dst)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 46.0±23.1 MB/s, size: 106.9 KB)


val: Scanning /content/drive/MyDrive/yolov8_project/datasets/VOC/labels/test2007.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [01:47<00:00,  2.89it/s]


                   all       4952      12032      0.782      0.739      0.805      0.594
             aeroplane        204        285      0.896      0.789      0.886      0.642
               bicycle        239        337      0.903      0.742      0.888      0.654
                  bird        282        459      0.799      0.678      0.786      0.531
                  boat        172        263      0.701      0.631      0.706      0.465
                bottle        212        469      0.763      0.691      0.719       0.49
                   bus        174        213      0.872      0.789      0.893      0.769
                   car        721       1201      0.851      0.868      0.916      0.729
                   cat        322        358      0.833       0.81      0.876      0.685
                 chair        417        756      0.591      0.606      0.601       0.41
                   cow        127        244      0.752      0.722      0.801      0.594
           diningtabl